In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import pandas as pd

# Load your data
df = pd.read_csv('../experiment_results/Size_full/results.csv')

# Exclude error_probability == 0.008
df = df[~((df["code"] == "gross") & (df["error_probability"] == 0.008))]

# Compute average logical error rate per code and error_type
avg_df = df.groupby(["code", "error_type"])["logical_error_rate"].mean().reset_index()
avg_df.rename(columns={"logical_error_rate": "avg_logical_error_rate"}, inplace=True)

# Merge back to compute absolute deviation
df = df.merge(avg_df, on=["code", "error_type"])
df["deviation"] = df["logical_error_rate"] - df["avg_logical_error_rate"]
df["abs_deviation"] = df["deviation"].abs()  # absolute deviation

# Find backend size with maximum deviation for each code and error_type
max_dev_idx = df.groupby(["code", "error_type"])["abs_deviation"].idxmax()
max_dev_df = df.loc[max_dev_idx, ["code", "error_type", "backend_size", "logical_error_rate", "deviation", "abs_deviation"]]

# Compute average absolute deviation per code and error_type
avg_dev_df = df.groupby(["code", "error_type"])["abs_deviation"].mean().reset_index()
avg_dev_df.rename(columns={"abs_deviation": "avg_abs_deviation"}, inplace=True)

# Print separate tables per error type
error_types = max_dev_df["error_type"].unique()
for e_type in error_types:
    table = max_dev_df[max_dev_df["error_type"] == e_type].sort_values("code")
    print(f"\n=== Error type: {e_type} ===")
    print(table[["code", "backend_size", "logical_error_rate", "deviation", "abs_deviation"]])

    # Also print average deviation per code
    avg_dev_table = avg_dev_df[avg_dev_df["error_type"] == e_type].sort_values("code")
    print("\nAverage absolute deviation per code:")
    print(avg_dev_table[["code", "avg_abs_deviation"]])



=== Error type: constant ===
        code  backend_size  logical_error_rate  deviation  abs_deviation
35     bacon           300               0.530   0.272000       0.272000
161    color           400               0.492   0.270333       0.270333
81     gross           400               0.001  -0.015000       0.015000
114       hh           300               0.147  -0.224533       0.224533
104   steane           500               0.443   0.139667       0.139667
143  surface           400               0.452   0.310733       0.310733

Average absolute deviation per code:
       code  avg_abs_deviation
0     bacon           0.181600
2     color           0.172089
4     gross           0.010500
6        hh           0.142089
8    steane           0.085956
10  surface           0.184089

=== Error type: modsi1000 ===
        code  backend_size  logical_error_rate  deviation  abs_deviation
62     bacon           300               0.014  -0.302467       0.302467
163    color           400 

In [3]:
# --- Cell 2: List codes with good logical error rate at p=0.002 ---

# Filter for error_probability = 0.002
filtered_df = df[df["error_probability"] == 0.002]

# Group by code and error_type and find which have logical_error_rate <= 0.002
good_df = (
    filtered_df[filtered_df["logical_error_rate"] <= 0.002]
    .sort_values(["error_type", "logical_error_rate"])
)

# Print results per error type
error_types = good_df["error_type"].unique()

for e_type in error_types:
    subset = good_df[good_df["error_type"] == e_type]
    print(f"\n=== Error type: {e_type} ===")
    if subset.empty:
        print("No codes found with logical_error_rate ≤ 0.002 at p=0.002.")
        continue
    display(
        subset[["code", "backend_size", "decoder", "logical_error_rate"]]
        if "decoder" in subset.columns
        else subset[["code", "backend_size", "logical_error_rate"]]
    )



=== Error type: constant ===


,code,backend_size,decoder,logical_error_rate
108,surface,300,bposd,0.000
109,surface,350,bposd,0.000
130,surface,400,bposd,0.000
132,surface,450,bposd,0.000
152,surface,500,bposd,0.000
61,bacon,350,bposd,0.001
63,bacon,400,bposd,0.001
81,gross,400,bposd,0.001
60,bacon,300,bposd,0.002
65,bacon,500,bposd,0.002



=== Error type: modsi1000 ===


,code,backend_size,decoder,logical_error_rate
110,surface,300,bposd,0.000
111,surface,350,bposd,0.000
134,surface,400,bposd,0.000
131,surface,450,bposd,0.001
153,surface,500,bposd,0.001


In [18]:
import pandas as pd

df = pd.read_csv('../experiment_results/Connectivity_small/results.csv')

# Define custom backend order
backend_order = ['custom_grid', 'custom_cube', 'custom_full']
df['backend'] = pd.Categorical(df['backend'], categories=backend_order, ordered=True)

# Sort by code and backend
df_sorted = df.sort_values(by=['code', 'backend']).reset_index(drop=True)

# Compute drop from cube to full for each code
cube_to_full_drops = []
for code, group in df_sorted.groupby('code'):
    group = group.sort_values('backend').reset_index(drop=True)
    # Only compute if both cube and full exist
    if {'custom_cube', 'custom_full'}.issubset(set(group['backend'])):
        cube_rate = group.loc[group['backend'] == 'custom_cube', 'logical_error_rate'].values[0]
        full_rate = group.loc[group['backend'] == 'custom_full', 'logical_error_rate'].values[0]
        drop_pct = (cube_rate - full_rate) / cube_rate * 100
        cube_to_full_drops.append({'code': code, 'cube_to_full_drop_pct': drop_pct})

df_cube_to_full = pd.DataFrame(cube_to_full_drops)

print(df_cube_to_full)
avg_drop = df_cube_to_full['cube_to_full_drop_pct'].mean()
print(f"\nAverage cube→full drop: {avg_drop:.2f}%")


      code  cube_to_full_drop_pct
0    bacon              86.538462
1    color              95.873786
2    gross              98.398398
3       hh              71.818182
4   steane              50.000000
5  surface             100.000000

Average cube→full drop: 83.77%


In [19]:
import pandas as pd

df = pd.read_csv('../experiment_results/Connectivity_small/results.csv')

# Define custom backend order
backend_order = ['custom_grid', 'custom_cube', 'custom_full']
df['backend'] = pd.Categorical(df['backend'], categories=backend_order, ordered=True)

# Sort by code and backend
df_sorted = df.sort_values(by=['code', 'backend']).reset_index(drop=True)

# Compute drop from grid to full for each code
grid_to_full_drops = []
for code, group in df_sorted.groupby('code'):
    group = group.sort_values('backend').reset_index(drop=True)
    # Only compute if both grid and full exist
    if {'custom_grid', 'custom_full'}.issubset(set(group['backend'])):
        grid_rate = group.loc[group['backend'] == 'custom_grid', 'logical_error_rate'].values[0]
        full_rate = group.loc[group['backend'] == 'custom_full', 'logical_error_rate'].values[0]
        drop_pct = (grid_rate - full_rate) / grid_rate * 100
        grid_to_full_drops.append({'code': code, 'grid_to_full_drop_pct': drop_pct})

df_grid_to_full = pd.DataFrame(grid_to_full_drops)

print(df_grid_to_full)
avg_drop = df_grid_to_full['grid_to_full_drop_pct'].mean()
print(f"\nAverage grid→full drop: {avg_drop:.2f}%")


      code  grid_to_full_drop_pct
0    bacon              85.858586
1    color              96.523517
2    gross              98.400000
3       hh              72.197309
4   steane              17.000000
5  surface             100.000000

Average grid→full drop: 78.33%


In [4]:
import pandas as pd

df = pd.read_csv('../experiment_results/Connectivity_small/results.csv')

# Define custom backend order
backend_order = ['custom_grid', 'custom_cube', 'custom_full']
df['backend'] = pd.Categorical(df['backend'], categories=backend_order, ordered=True)

# Sort by code and backend
df_sorted = df.sort_values(by=['code', 'backend']).reset_index(drop=True)

# Compute drop from cube to full for each code
grid_to_cube_drops = []
for code, group in df_sorted.groupby('code'):
    group = group.sort_values('backend').reset_index(drop=True)
    # Only compute if both cube and full exist
    if {'custom_cube', 'custom_full'}.issubset(set(group['backend'])):
        cube_rate = group.loc[group['backend'] == 'custom_cube', 'logical_error_rate'].values[0]
        grid_rate = group.loc[group['backend'] == 'custom_grid', 'logical_error_rate'].values[0]
        drop_pct = (grid_rate - cube_rate) / grid_rate * 100
        grid_to_cube_drops.append({'code': code, 'cube_to_full_drop_pct': drop_pct})

df_grid_to_cube = pd.DataFrame(grid_to_cube_drops)

print(df_grid_to_cube)


      code  cube_to_full_drop_pct
0    bacon              -5.050505
1    color              15.746421
2    gross               0.100000
3       hh               1.345291
4   steane             -66.000000
5  surface              28.776978


In [23]:
import pandas as pd

# Load and sort data
df = pd.read_csv('../experiment_results/Variance/results.csv')
backend_order = ['variance_low', 'variance_mid', 'variance_high']
df['backend'] = pd.Categorical(df['backend'], categories=backend_order, ordered=True)
df_sorted = df.sort_values(by=['code', 'backend']).reset_index(drop=True)

# Group by code and backend to get mean logical error rate
error_summary = df_sorted.groupby(['code', 'backend'])['logical_error_rate'].mean().reset_index()

# Pivot so backends are columns
error_pivot = error_summary.pivot(index='code', columns='backend', values='logical_error_rate')

# Calculate differences between backends
error_pivot['mid_vs_low'] = error_pivot['variance_mid'] - error_pivot['variance_low']
error_pivot['high_vs_mid'] = error_pivot['variance_high'] - error_pivot['variance_mid']
error_pivot['high_vs_low'] = error_pivot['variance_high'] - error_pivot['variance_low']

print(error_pivot)


backend  variance_low  variance_mid  variance_high  mid_vs_low  high_vs_mid  \
code                                                                          
bacon           0.487         0.493          0.491       0.006       -0.002   
color           0.498         0.486          0.508      -0.012        0.022   
gross           1.000         1.000          0.999       0.000       -0.001   
hh              0.497         0.504          0.513       0.007        0.009   
steane          0.023         0.023          0.016       0.000       -0.007   
surface         0.512         0.498          0.518      -0.014        0.020   

backend  high_vs_low  
code                  
bacon          0.004  
color          0.010  
gross         -0.001  
hh             0.016  
steane        -0.007  
surface        0.006  


/tmp/ipykernel_26426/3272860985.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  error_summary = df_sorted.groupby(['code', 'backend'])['logical_error_rate'].mean().reset_index()


In [25]:
# Assuming error_pivot already exists with variance_low/mid/high columns

# Compute percentage differences
error_pivot['mid_vs_low_%'] = ((error_pivot['variance_mid'] - error_pivot['variance_low']) 
                               / error_pivot['variance_low'] * 100)

error_pivot['high_vs_mid_%'] = ((error_pivot['variance_high'] - error_pivot['variance_mid']) 
                                / error_pivot['variance_mid'] * 100)

error_pivot['high_vs_low_%'] = ((error_pivot['variance_high'] - error_pivot['variance_low']) 
                                / error_pivot['variance_low'] * 100)

# Find maximum percentage difference per code
error_pivot['max_diff_%'] = error_pivot[['mid_vs_low_%', 'high_vs_mid_%', 'high_vs_low_%']].abs().max(axis=1)

# Code with largest percentage difference
biggest_diff_code = error_pivot['max_diff_%'].idxmax()
biggest_diff_value = error_pivot.loc[biggest_diff_code, 'max_diff_%']

print("Code with largest variance effect (percent):", biggest_diff_code)
print("Largest percentage difference in logical error rate: {:.2f}%".format(biggest_diff_value))

# Optional: sort by percentage difference
sorted_diffs = error_pivot.sort_values(by='max_diff_%', ascending=False)
print(sorted_diffs[['mid_vs_low_%', 'high_vs_mid_%', 'high_vs_low_%', 'max_diff_%']])


Code with largest variance effect (percent): steane
Largest percentage difference in logical error rate: 30.43%
backend  mid_vs_low_%  high_vs_mid_%  high_vs_low_%  max_diff_%
code                                                           
steane       0.000000     -30.434783     -30.434783   30.434783
color       -2.409639       4.526749       2.008032    4.526749
surface     -2.734375       4.016064       1.171875    4.016064
hh           1.408451       1.785714       3.219316    3.219316
bacon        1.232033      -0.405680       0.821355    1.232033
gross        0.000000      -0.100000      -0.100000    0.100000


In [ ]:
import pandas as pd

# Load and sort data
df = pd.read_csv('../experiment_results/Topology/results.csv')
df = df[df["backend"] != "real_infleqtion"]
df_sorted = df.sort_values(by=['code', 'backend']).reset_index(drop=True)

# Group by code and backend to get mean logical error rate
error_summary = df_sorted.groupby(['code', 'backend'])['logical_error_rate'].mean().reset_index()

# Pivot so backends are columns
error_pivot = error_summary.pivot(index='code', columns='backend', values='logical_error_rate')
print("Per-code averages:")
print(error_pivot)

# Average across codes for each backend
backend_means = error_pivot.mean(axis=0)
print("\nAverage per backend:")
print(backend_means)

# Combined averages
combined = {
    "willow+nsinfleqtion": error_pivot[['real_willow', 'real_nsinfleqtion']].mean(axis=1).mean(),
    "apollo": error_pivot[['real_apollo']].mean(axis=1).mean()
}
print("\nCombined averages:")
print(combined)

# General average per code across all available backends
per_code_avg = error_pivot.mean(axis=1)
print("\nGeneral average per code:")
print(per_code_avg)


Per-code averages:
backend  real_apollo  real_nsinfleqtion  real_willow
code                                                
bacon          0.518              0.482        0.496
color          0.172              0.497        0.522
gross          0.024              0.999        0.999
hh             0.162              0.507        0.486
steane         0.437              0.049        0.073
surface        0.000              0.480        0.490

Average per backend:
backend
real_apollo          0.218833
real_nsinfleqtion    0.502333
real_willow          0.511000
dtype: float64

Combined averages:
{'willow+nsinfleqtion': 0.5066666666666667, 'apollo': 0.21883333333333332}


In [ ]:
import pandas as pd
import os

path = "../experiment_results"

datasets_normal = [
    ("DQC", "DQC Full Size"),
    ("DQC_1_QPU", "DQC 1 QPU Size")
]

datasets_lower = [
    ("DQC_LOWER", "DQC Full Size"),
    ("DQC_1_QPU_LOWER", "DQC 1 QPU Size")
]

def compute_summary(datasets, label):
    dfs = []
    for folder, ds_label in datasets:
        tech_path = os.path.join(path, folder, "results.csv")
        df_tmp = pd.read_csv(tech_path)
        df_tmp["dataset"] = ds_label
        dfs.append(df_tmp)

    df = pd.concat(dfs, ignore_index=True)

    # Exclude gross
    df = df[df["code"].str.lower() != "gross"]

    # Group by code & dataset to compute mean error
    summary = df.groupby(["code", "dataset"])["logical_error_rate"].mean().reset_index()

    # Pivot so datasets become columns
    pivot = summary.pivot(index="code", columns="dataset", values="logical_error_rate")

    # Compute improvements
    pivot["abs_improvement"] = pivot["DQC Full Size"] - pivot["DQC 1 QPU Size"]
    pivot["pct_improvement"] = (pivot["abs_improvement"] / pivot["DQC Full Size"]) * 100
    pivot["group"] = label  # tag Normal vs Lower

    # Compute overall averages
    avg_abs = pivot["abs_improvement"].mean()
    avg_pct = pivot["pct_improvement"].mean()

    print(f"\n===== {label} Results =====")
    print(pivot.sort_values("pct_improvement", ascending=False))
    print("\nAverage absolute improvement:", avg_abs)
    print("Average percentage improvement:", avg_pct)

    return pivot

# Run for both sets
pivot_normal = compute_summary(datasets_normal, "Normal")
pivot_lower = compute_summary(datasets_lower, "Lower")

# Combine both for global averages
combined = pd.concat([pivot_normal, pivot_lower], ignore_index=True)

overall_avg_abs = combined["abs_improvement"].mean()
overall_avg_pct = combined["pct_improvement"].mean()

print("\n===== Overall Averages Across Normal + Lower =====")
print("Overall average absolute improvement:", overall_avg_abs)
print("Overall average percentage improvement:", overall_avg_pct)



===== Normal Results =====
dataset  DQC 1 QPU Size  DQC Full Size  abs_improvement  pct_improvement
code                                                                    
surface           0.033          0.481            0.448        93.139293
color             0.191          0.502            0.311        61.952191
hh                0.192          0.496            0.304        61.290323
steane            0.063          0.079            0.016        20.253165
bacon             0.420          0.513            0.093        18.128655

Average absolute improvement: 0.2344
Average percentage improvement: 50.95272529654406

===== Lower Results =====
dataset  DQC 1 QPU Size  DQC Full Size  abs_improvement  pct_improvement
code                                                                    
surface           0.009          0.301            0.292        97.009967
hh                0.119          0.439            0.320        72.892938
color             0.183          0.503            0.32

In [ ]:
import pandas as pd

# Path to the single CSV file
path = "../experiment_results/DQC_Nighthawk/results.csv"
df = pd.read_csv(path)

df = pd.read_csv(path)

# Exclude gross and routing_method = basic
df = df[(df["code"].str.lower() != "gross") & (df["routing_method"].str.lower() != "basic")]

# Map backend to dataset labels
dataset_map = {
    "real_nighthawk": "DQC Full Size",
    "real_nighthawk_1_qpu": "DQC 1 QPU Size"
}
df["dataset"] = df["backend"].map(dataset_map)

# Map error_probability to groups
group_map = {1: "Normal", 10: "Lower"}
df["group"] = df["error_probability"].map(group_map)

# Group by code, dataset, and group to compute mean error
summary = df.groupby(["code", "dataset", "group"])["logical_error_rate"].mean().reset_index()

# Iterate over groups (Normal, Lower)
for grp, subdf in summary.groupby("group"):
    pivot = subdf.pivot(index="code", columns="dataset", values="logical_error_rate")

    # Compute improvements
    pivot["abs_improvement"] = pivot["DQC Full Size"] - pivot["DQC 1 QPU Size"]
    if pivot["DQC Full Size"] != 0:
        pivot["pct_improvement"] = (pivot["abs_improvement"] / pivot["DQC Full Size"]) * 100
    else:
        pivot["pct_improvement"]
    print(f"\n===== {grp} Results =====")
    print(pivot.sort_values("pct_improvement", ascending=False))

    # Group averages
    avg_abs = pivot["abs_improvement"].mean()
    avg_pct = pivot["pct_improvement"].mean()
    print("\nAverage absolute improvement:", avg_abs)
    print("Average percentage improvement:", avg_pct)


===== Lower Results =====
dataset  DQC 1 QPU Size  DQC Full Size  abs_improvement  pct_improvement
code                                                                    
bacon             0.052          0.522            0.470        90.038314
hh                0.003          0.002           -0.001       -50.000000
color             0.006          0.003           -0.003      -100.000000
steane            0.048          0.021           -0.027      -128.571429
surface           0.001          0.000           -0.001             -inf

Average absolute improvement: 0.0876
Average percentage improvement: -inf

===== Normal Results =====
dataset  DQC 1 QPU Size  DQC Full Size  abs_improvement  pct_improvement
code                                                                    
bacon             0.155          0.524            0.369        70.419847
color             0.005          0.014            0.009        64.285714
steane            0.061          0.058           -0.003        -5.1

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# -----------------------
# Load all backend datasets
# -----------------------

# Backend 1 (combined from two files)
df1 = pd.read_csv("../experiment_results/Willow/results.csv")
df2 = pd.read_csv("../experiment_results/Infleqtion/results.csv")
df3 = pd.read_csv("../experiment_results/Apollo/results.csv")
df = pd.concat([df1, df2, df3], ignore_index=True)

backend_avg = df.groupby("backend")["logical_error_rate"].mean().reset_index()

# -----------------------
# Average per code
# -----------------------
code_avg = df.groupby("code")["logical_error_rate"].mean().reset_index()

print("Average per backend:\n", backend_avg)
print("\nAverage per code:\n", code_avg)

print(f"NS: {(0.000833 + 0.496667)/2}")
print(f"S: {(0.005667 + 0.244667)/2}")


Average per backend:
              backend  logical_error_rate
0        real_apollo            0.005667
1    real_infleqtion            0.244667
2      real_nsapollo            0.000833
3  real_nsinfleqtion            0.496667
4        real_willow            0.567500

Average per code:
       code  logical_error_rate
0    bacon              0.2940
1    color              0.2028
2    gross              0.5776
3       hh              0.2142
4   steane              0.0900
5  surface              0.1998
NS: 0.24875
S: 0.125167


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# -----------------------
# Load backend dataset
# -----------------------
df = pd.read_csv("../experiment_results/Routing_grid/results.csv")

# -----------------------
# Mean + variance per backend
# -----------------------
backend_stats = (
    df.groupby("backend", as_index=False)
      .agg(
          gate_overhead_mean=("swap_overhead_mean", "mean"),
          gate_overhead_var=("swap_overhead_var", "mean")  # assumes column exists
      )
)

# -----------------------
# Average per code
# -----------------------
code_avg = df.groupby("code")["swap_overhead_mean"].mean().reset_index()

# -----------------------
# Average per routing method
# -----------------------
routing_avg = df.groupby("routing_method")["swap_overhead_mean"].mean().reset_index()

# -----------------------
# Average per routing + layout method combination
# -----------------------
combo_avg = (
    df.groupby(["routing_method", "layout_method"], as_index=False)
      .agg(
          gate_overhead_mean=("swap_overhead_mean", "mean"),
          gate_overhead_var=("swap_overhead_var", "mean")
      )
)

# Pivot table form (just for means)
combo_pivot = (
    df.pivot_table(
        index="routing_method",
        columns="layout_method",
        values="swap_overhead_mean",
        aggfunc="mean"
    )
)

# -----------------------
# Best routing+layout per code
# -----------------------
# Compute mean overhead per code & combination
code_combo = (
    df.groupby(["code", "routing_method", "layout_method"], as_index=False)
      .agg(best_gate_overhead=("swap_overhead_mean", "mean"))
)

# For each code, find the row with the minimum swap_overhead_mean
best_per_code = (
    code_combo.loc[code_combo.groupby("code")["best_gate_overhead"].idxmin()]
    .reset_index(drop=True)
)

# Rename columns for clarity
best_per_code.rename(columns={
    "routing_method": "best_routing",
    "layout_method": "best_layout",
    "best_gate_overhead": "best_value"
}, inplace=True)

# Make sure pandas prints everything
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# -----------------------
# Print all results
# -----------------------
print("Average per backend (mean + var):\n", backend_stats)
print("\nAverage per code:\n", code_avg)
print("\nAverage per routing method:\n", routing_avg)
print("\nAverage per routing+layout combination (mean + var):\n", combo_avg)
print("\nBest routing+layout combination per code:\n", best_per_code)


Average per backend (mean + var):
        backend  gate_overhead_mean  gate_overhead_var
0  custom_grid        25080.044815      385852.408816

Average per code:
       code  swap_overhead_mean
0    bacon         1197.249778
1    color        18329.354333
2    gross        82506.326667
3       hh        18650.467556
4   steane         9158.226000
5  surface        20638.644556

Average per routing method:
   routing_method  swap_overhead_mean
0          basic        34703.670889
1          sabre        15231.403722
2     stochastic        25305.059833

Average per routing+layout combination (mean + var):
   routing_method layout_method  gate_overhead_mean  gate_overhead_var
0          basic         dense        37831.333333       0.000000e+00
1          basic         sabre        30779.512667       1.076677e+06
2          basic       trivial        35500.166667       0.000000e+00
3          sabre         dense        18312.346833       1.415587e+05
4          sabre         sabre       

In [4]:
import pandas as pd
import numpy as np

# -----------------------
# Load routing dataset
# -----------------------
#df = pd.read_csv("../experiment_results/Routing_grid/results.csv")
df = pd.read_csv("../experiment_results/Routing_hh/results.csv")  # change this path as needed

# Merge into one dataset
#df = pd.concat([df1, df2], ignore_index=True)


# -----------------------
# Load tq_gates_total from Translation results
# -----------------------
df_gates = pd.read_csv("../experiment_results/Translation/results.csv")

# Keep only relevant columns: code + original_2q_gates
df_gates = df_gates[["code", "original_2q_gates"]].drop_duplicates()

# -----------------------
# Merge on code
# -----------------------
df = df.merge(df_gates, on="code", how="left")

# -----------------------
# Add normalized percentage value
# -----------------------
df["swap_overhead_pct"] = (df["swap_overhead_mean"] / df["original_2q_gates"]) * 100

# -----------------------
# Mean + variance per backend
# -----------------------
backend_stats = (
    df.groupby("backend", as_index=False)
      .agg(
          gate_overhead_mean=("swap_overhead_mean", "mean"),
          gate_overhead_var=("swap_overhead_var", "mean"),
          gate_overhead_pct=("swap_overhead_pct", "mean")
      )
)

# -----------------------
# Average per code (percentage)
# -----------------------
code_avg = (
    df.groupby("code", as_index=False)
      .agg(
          raw_mean=("swap_overhead_mean", "mean"),
          norm_pct=("swap_overhead_pct", "mean")
      )
)

# -----------------------
# Average per routing method (percentage)
# -----------------------
routing_avg = (
    df.groupby("routing_method", as_index=False)
      .agg(
          raw_mean=("swap_overhead_mean", "mean"),
          norm_pct=("swap_overhead_pct", "mean")
      )
)

# -----------------------
# Average per routing + layout method combination (percentage)
# -----------------------
combo_avg = (
    df.groupby(["routing_method", "layout_method"], as_index=False)
      .agg(
          raw_mean=("swap_overhead_mean", "mean"),
          raw_var=("swap_overhead_var", "mean"),
          norm_pct=("swap_overhead_pct", "mean")
      )
)

# Pivot table form (just normalized percentages)
combo_pivot = (
    df.pivot_table(
        index="routing_method",
        columns="layout_method",
        values="swap_overhead_pct",
        aggfunc="mean"
    )
)

# -----------------------
# Best routing+layout per code (percentage criterion)
# -----------------------
code_combo = (
    df.groupby(["code", "routing_method", "layout_method"], as_index=False)
      .agg(
          raw_mean=("swap_overhead_mean", "mean"),
          norm_pct=("swap_overhead_pct", "mean")
      )
)

best_per_code = (
    code_combo.loc[code_combo.groupby("code")["norm_pct"].idxmin()]
    .reset_index(drop=True)
)

best_per_code.rename(columns={
    "routing_method": "best_routing",
    "layout_method": "best_layout",
    "norm_pct": "best_norm_pct",
    "raw_mean": "best_raw_value"
}, inplace=True)

# -----------------------
# Print results
# -----------------------
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

print("Average per backend (mean + var + %):\n", backend_stats)
print("\nAverage per code (raw + %):\n", code_avg)
print("\nAverage per routing method (raw + %):\n", routing_avg)
print("\nAverage per routing+layout combination (raw + var + %):\n", combo_avg)
print("\nBest routing+layout combination per code (% criterion):\n", best_per_code)


Average per backend (mean + var + %):
            backend  gate_overhead_mean  gate_overhead_var  gate_overhead_pct
0  custom_heavyhex          12305.3332       245729.00557         317.508386

Average per code (raw + %):
       code      raw_mean    norm_pct
0    bacon   2396.845889  220.298335
1    color  12581.235333  238.280972
2       hh  18245.859111  259.174135
3   steane  13580.108222  565.602175
4  surface  14722.617444  304.186311

Average per routing method (raw + %):
   routing_method      raw_mean    norm_pct
0          basic  16679.556667  417.378770
1          sabre   8341.445000  222.472302
2     stochastic  11894.997933  312.674085

Average per routing+layout combination (raw + var + %):
   routing_method layout_method    raw_mean       raw_var    norm_pct
0          basic         dense  17251.4000  0.000000e+00  423.991566
1          basic         sabre  12508.6700  1.068834e+06  317.081487
2          basic       trivial  20278.6000  0.000000e+00  511.063258
3        

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# -----------------------
# Load datasets (two files)
# -----------------------
#df = pd.read_csv("../experiment_results/Routing_grid/results.csv")
df = pd.read_csv("../experiment_results/Routing_hh/results.csv")  # change this path as needed

# Merge into one dataset
#df = pd.concat([df1, df2], ignore_index=True)

# -----------------------
# Mean + variance per backend
# -----------------------
backend_stats = (
    df.groupby("backend", as_index=False)
      .agg(
          gate_overhead_mean=("swap_overhead_mean", "mean"),
          gate_overhead_var=("swap_overhead_var", "mean")  # assumes column exists
      )
)

# -----------------------
# Average per code (across all backends)
# -----------------------
code_avg = df.groupby("code")["swap_overhead_mean"].mean().reset_index()

# -----------------------
# Average per routing method (across all backends)
# -----------------------
routing_avg = df.groupby("routing_method")["swap_overhead_mean"].mean().reset_index()

# -----------------------
# Average per routing + layout method combination
# -----------------------
combo_avg = (
    df.groupby(["routing_method", "layout_method"], as_index=False)
      .agg(
          gate_overhead_mean=("swap_overhead_mean", "mean"),
          gate_overhead_var=("swap_overhead_var", "mean")
      )
)

# Pivot table form (just for means)
combo_pivot = (
    df.pivot_table(
        index="routing_method",
        columns="layout_method",
        values="swap_overhead_mean",
        aggfunc="mean"
    )
)

# -----------------------
# Best routing+layout per code (across all backends)
# -----------------------
code_combo = (
    df.groupby(["code", "routing_method", "layout_method"], as_index=False)
      .agg(best_gate_overhead=("swap_overhead_mean", "mean"))
)

best_per_code = (
    code_combo.loc[code_combo.groupby("code")["best_gate_overhead"].idxmin()]
    .reset_index(drop=True)
)

best_per_code.rename(columns={
    "routing_method": "best_routing",
    "layout_method": "best_layout",
    "best_gate_overhead": "best_value"
}, inplace=True)

# -----------------------
# Print all results
# -----------------------
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

general_avg = df["swap_overhead_mean"].mean()
print("\nGeneral average across all codes and backends:", general_avg)
sabre_avg = df[
    (df["routing_method"] == "sabre") & (df["layout_method"] == "sabre")
]["swap_overhead_mean"].mean()

print("\nAverage swap_overhead_mean for sabre x sabre combination:", sabre_avg)

sabre_hh_custom = df[
    (df["code"] == "hh") &
    (df["backend"] == "custom_heavyhex") &
    (df["routing_method"] == "sabre") &
    (df["layout_method"] == "sabre")
]["swap_overhead_mean"].mean()

print("\nAverage swap_overhead_mean for sabre x sabre, code=hh, backend=custom_heavyhex:", sabre_hh_custom)


print("Average per backend (mean + var):\n", backend_stats)
print("\nAverage per code (across all backends):\n", code_avg)
print("\nAverage per routing method (across all backends):\n", routing_avg)
print("\nAverage per routing+layout combination (mean + var):\n", combo_avg)
print("\nBest routing+layout combination per code (across all backends):\n", best_per_code)



General average across all codes and backends: 12305.3332

Average swap_overhead_mean for sabre x sabre combination: 4789.2756

Average swap_overhead_mean for sabre x sabre, code=hh, backend=custom_heavyhex: 6201.596
Average per backend (mean + var):
            backend  gate_overhead_mean  gate_overhead_var
0  custom_heavyhex          12305.3332       245729.00557

Average per code (across all backends):
       code  swap_overhead_mean
0    bacon         2396.845889
1    color        12581.235333
2       hh        18245.859111
3   steane        13580.108222
4  surface        14722.617444

Average per routing method (across all backends):
   routing_method  swap_overhead_mean
0          basic        16679.556667
1          sabre         8341.445000
2     stochastic        11894.997933

Average per routing+layout combination (mean + var):
   routing_method layout_method  gate_overhead_mean  gate_overhead_var
0          basic         dense          17251.4000       0.000000e+00
1       

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# -----------------------
# Load backend dataset
# -----------------------
df = pd.read_csv("../experiment_results/Translation/results.csv")

# -----------------------
# Mean + variance per backend
# -----------------------
backend_stats = (
    df.groupby("backend", as_index=False)
      .agg(
          gate_overhead_mean=("gate_overhead_mean", "mean"),
          gate_overhead_var=("gate_overhead_var", "mean")  # assumes column exists
      )
)

# -----------------------
# Average per code
# -----------------------
code_avg = df.groupby("code")["gate_overhead_mean"].mean().reset_index()

# -----------------------
# Average per routing method
# -----------------------
translating_avg = df.groupby("translating_method")["gate_overhead_mean"].mean().reset_index()

print(translating_avg)


  translating_method  gate_overhead_mean
0             qiskit        27552.000000
1   qiskit_optimized        21507.750000
2               tket        56552.416667
3     tket_optimized        55592.166667


In [16]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv("../experiment_results/Translation/results.csv")

# Compute per-row normalized gate overhead
df["normalized_overhead"] = df["gate_overhead_mean"] / df["original_total_gates"]

# -----------------------
# Average per code
# -----------------------
code_avg = df.groupby("code")["normalized_overhead"].mean().reset_index()
code_avg.rename(columns={"normalized_overhead": "avg_normalized_overhead"}, inplace=True)

# -----------------------
# Average per translating method
# -----------------------
translating_avg = df.groupby("translating_method")["normalized_overhead"].mean().reset_index()
translating_avg.rename(columns={"normalized_overhead": "avg_normalized_overhead"}, inplace=True)

# -----------------------
# Average per backend
# -----------------------
backend_avg = df.groupby("backend")["normalized_overhead"].mean().reset_index()
backend_avg.rename(columns={"normalized_overhead": "avg_normalized_overhead"}, inplace=True)

print("Per code:")
print(code_avg)

print("\nPer translating method:")
print(translating_avg)

print("\nPer backend:")
print(backend_avg)


Per code:
      code  avg_normalized_overhead
0    bacon                 5.301020
1    color                 6.504127
2    gross                 6.631657
3       hh                 6.592105
4   steane                 5.509372
5  surface                 6.267857

Per translating method:
  translating_method  avg_normalized_overhead
0             qiskit                 4.260593
1   qiskit_optimized                 3.166202
2               tket                 8.704297
3     tket_optimized                 8.406333

Per backend:
       backend  avg_normalized_overhead
0  custom_full                 6.134356


In [17]:
# Compute mean per code (already done)
code_avg = df.groupby("code")["normalized_overhead"].mean().reset_index()

# Compute global average normalized overhead
global_avg = df["normalized_overhead"].mean()

# Add proportional and deviation columns
code_avg["proportional_overhead"] = code_avg["normalized_overhead"] / global_avg
code_avg["deviation_from_average"] = code_avg["proportional_overhead"] - 1

print(code_avg)


      code  normalized_overhead  proportional_overhead  deviation_from_average
0    bacon             5.301020               0.864153               -0.135847
1    color             6.504127               1.060279                0.060279
2    gross             6.631657               1.081068                0.081068
3       hh             6.592105               1.074621                0.074621
4   steane             5.509372               0.898117               -0.101883
5  surface             6.267857               1.021763                0.021763


In [13]:

from qiskit.transpiler import CouplingMap
import math
d = (2 + math.sqrt(40 * 291 + 24)) / 10
d = int(d)
if d % 2 == 0:
    d -= 1
print(d)
coupling_map = CouplingMap.from_heavy_hex(d)
print(coupling_map.size())

11
291


In [6]:
# === General decoders summary with best decoder per code ===
import pandas as pd

# Load CSV
df_general = pd.read_csv("../experiment_results/Decoders/results_general.csv")


# Filter only modsi1000 results and exclude gross code
df_general = df_general[(df_general["error_type"] == "constant")] #& (df_general["code"].str.lower() != "gross")]
print(df_general)

# Optional renaming
if "decoder_map" in globals():
    df_general["decoder"] = df_general["decoder"].apply(lambda x: decoder_map.get(x, x))
if "code_rename_map" in globals():
    df_general["code"] = df_general["code"].apply(lambda x: code_rename_map.get(x.lower(), x.capitalize()))
else:
    df_general["code"] = df_general["code"].apply(lambda x: x.capitalize())

# Compute mean per decoder
summary_general = (
    df_general
    .groupby("decoder", as_index=False)
    .agg({
        "logical_error_rate": "mean",
        "exec_time": "mean"
    })
    .sort_values("decoder")
)

print("=== General Decoders Average (modsi1000 only, excluding 'gross') ===")
display(summary_general)

# Determine best decoder per code
best_per_code = (
    df_general
    .groupby("code")
    .apply(lambda x: x.loc[x["logical_error_rate"].idxmin()])
    .reset_index(drop=True)
    [["code", "decoder", "logical_error_rate", "exec_time"]]
)

print("=== Best Decoder per Code (lowest logical error rate) ===")
display(best_per_code)


        backend  backend_size     code      decoder  distance  cycles  \
1   custom_full           400   steane         mwpm         9       9   
2   custom_full           400  surface         mwpm        13      13   
3   custom_full           400       hh         mwpm        11      11   
4   custom_full           400    color         mwpm        13      13   
9   custom_full           400    bacon         mwpm        19      19   
10  custom_full           400    gross    bposd_chk        12      12   
11  custom_full           400    gross  bposd_batch        12      12   
14  custom_full           400   steane    bposd_chk         9       9   
15  custom_full           400   steane  bposd_batch         9       9   
20  custom_full           400       hh    bposd_chk        11      11   
21  custom_full           400       hh  bposd_batch        11      11   
22  custom_full           400  surface    bposd_chk        13      13   
24  custom_full           400  surface  bposd_batch

,decoder,logical_error_rate,exec_time
0,bposd_batch,0.312333,27755.632234
1,bposd_chk,0.308500,27980.924723
2,mwpm,0.359800,0.383000


=== Best Decoder per Code (lowest logical error rate) ===


/tmp/ipykernel_64019/3767896263.py:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.loc[x["logical_error_rate"].idxmin()])


,code,decoder,logical_error_rate,exec_time
0,Bacon,bposd_chk,0.494,39613.959832
1,Color,bposd_batch,0.243,59967.093388
2,Gross,bposd_chk,0.283,4592.632116
3,Hh,bposd_chk,0.477,20436.631934
4,Steane,mwpm,0.286,0.175250
5,Surface,bposd_chk,0.022,34813.058622


In [7]:
import pandas as pd

# Assuming df_general is already filtered for modsi1000 and excludes "gross"

def best_worst_gap(group):
    # Sort decoders by logical_error_rate
    sorted_rates = group.sort_values("logical_error_rate")
    best_rate = sorted_rates.iloc[0]["logical_error_rate"]
    worst_rate = sorted_rates.iloc[-1]["logical_error_rate"]
    gap = worst_rate - best_rate
    best_decoder = sorted_rates.iloc[0]["decoder"]
    worst_decoder = sorted_rates.iloc[-1]["decoder"]
    
    return pd.Series({
        "best_decoder": best_decoder,
        "best_rate": best_rate,
        "worst_decoder": worst_decoder,
        "worst_rate": worst_rate,
        "gap": gap
    })

# Apply per code
gap_df = df_general.groupby("code").apply(best_worst_gap).reset_index()

# Find code with the largest gap
max_gap_row = gap_df.loc[gap_df["gap"].idxmax()]

print("=== Gap between best and worst decoder per code ===")
display(gap_df)

print("\nCode with the largest difference between best and worst decoder:")
display(max_gap_row)


=== Gap between best and worst decoder per code ===


/tmp/ipykernel_64019/1188751606.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gap_df = df_general.groupby("code").apply(best_worst_gap).reset_index()


,code,best_decoder,best_rate,worst_decoder,worst_rate,gap
0,Bacon,bposd_chk,0.494,mwpm,0.515,0.021
1,Color,bposd_batch,0.243,mwpm,0.487,0.244
2,Gross,bposd_chk,0.283,bposd_batch,0.302,0.019
3,Hh,bposd_chk,0.477,bposd_batch,0.485,0.008
4,Steane,mwpm,0.286,bposd_chk,0.326,0.040
5,Surface,bposd_chk,0.022,bposd_batch,0.033,0.011



Code with the largest difference between best and worst decoder:


code                   Color
best_decoder     bposd_batch
best_rate              0.243
worst_decoder           mwpm
worst_rate             0.487
gap                    0.244
Name: 1, dtype: object

In [8]:
# === Special decoders summary ===
import pandas as pd

# Load your CSV
df_special = pd.read_csv("../experiment_results/Decoders/results_special.csv")

# Filter only modsi1000 results
df_special = df_special[df_special["error_type"] == "phenomenological"]

# Optional renaming (if maps defined in globals)
if "decoder_map" in globals():
    df_special["decoder"] = df_special["decoder"].apply(lambda x: decoder_map.get(x, x))
if "code_rename_map" in globals():
    df_special["code"] = df_special["code"].apply(lambda x: code_rename_map.get(x.lower(), x.capitalize()))
else:
    df_special["code"] = df_special["code"].apply(lambda x: x.capitalize())

# Compute mean logical error rate and execution time per decoder and per code
summary_special = (
    df_special
    .groupby(["decoder", "code"], as_index=False)
    .agg({
        "logical_error_rate": "mean"
        
    })
    .sort_values(["decoder", "code"])
)

# Display nicely
print("=== Special Decoders Summary (modsi1000 only) ===")
display(summary_special)


=== Special Decoders Summary (modsi1000 only) ===


,decoder,code,logical_error_rate
0,bposd,Color,0.003633
1,chromobius,Color,0.001100
